In [0]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print(tf.__version__)

In [0]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)



In [0]:
print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

# EXPECTED OUTPUT
# train_images.shape: (60000, 28, 28, 1), of float64
# test_images.shape: (10000, 28, 28, 1), of float64

In [0]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

In [0]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

print('\nSaved model:')
!ls -l {export_path}

In [0]:
!saved_model_cli show --dir {export_path} --all

# EXPECTED OUTPUT
# MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:
#
# signature_def['serving_default']:
#   The given SavedModel SignatureDef contains the following input(s):
#     inputs['input_image'] tensor_info:
#        dtype: DT_FLOAT
#        shape: (-1, 28, 28, 1)
#        name: Conv1_input:0
#  The given SavedModel SignatureDef contains the following output(s):
#    outputs['Softmax/Softmax:0'] tensor_info:
#        dtype: DT_FLOAT
#        shape: (-1, 10)
#        name: Softmax/Softmax:0
#  Method name is: tensorflow/serving/predict

In [0]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server

In [0]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=digits_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

# Expected output
# Starting job # 0 in a separate thread.

In [0]:
!tail server.log

In [0]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

test_image = 42


show(test_image, 'An Example Image: {}'.format(test_labels[test_image]))

# EXPECTED OUTPUT
# An example Image: 4, and a plot of the number 4

In [0]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))
print(data)

In [0]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/digits_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print(predictions)

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  np.argmax(predictions[0]), test_labels[0], np.argmax(predictions[0]), test_labels[0]))

# EXPECTED OUTPUT
# [[2.68937868e-07, 6.87597923e-10, 1.2811563e-06, 9.07639405e-05, 4.35213154e-09, 1.56858093e-09, 6.00412569e-13, 0.999889, 2.59646663e-06, 1.61339703e-05], [2.23855227e-06, 5.49161132e-06, 0.999600112, 2.19848971e-06, 1.30025773e-11, 1.28111242e-06, 0.000387982698, 1.64094575e-14, 6.61974241e-07, 7.12327549e-12], [4.6440724e-05, 0.996990204, 0.000780542672, 8.95627818e-05, 0.0008044485, 5.88553521e-05, 0.000464979181, 0.000534292951, 0.000190246137, 4.04530474e-05]]
#
# The model thought this was a 7 (class 7), and it was actually a 7 (class 7)
# An image plot of  number 7